In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import ast
import wfdb
from tensorflow import keras
from tensorflow import lite
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

print("Done")

Done


In [3]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = '../../../Datasets/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.1\\'

sampling_rate=100
calssificatin_type = "superclasses"    #{"binary","superclasses","subclasses"}

lead_types={"lead-I":[1,2,3,4,5,6,7,8,9,10,11], "bipolar-limb":[3,4,5,6,7,8,9,10,11] , "unipolar-limb":[0,1,2,6,7,8,9,10,11], "limb-leads":[6,7,8,9,10,11] , "precordial-leads":[0,1,2,3,4,5],"all-lead":[]}
lead_name="all-lead"

# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_superclass_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

def aggregate_subclass_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_subclass)
    ret = list(set(tmp))
    return ret

if calssificatin_type == "superclasses":
    Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_superclass_diagnostic)
else:
    Y['diagnostic_subclass'] = Y.scp_codes.apply(aggregate_subclass_diagnostic)
    
# Ensure y_train is correctly transformed
mlb = MultiLabelBinarizer(classes=['CD', 'HYP', 'MI', 'NORM', 'STTC'])
Y = mlb.fit_transform(Y.diagnostic_superclass)

print("Done")

Done


In [4]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1000, 1)),
        tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv1D(128, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv1D(256, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv1D(512, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(5, activation='sigmoid')  # Output layer with 5 elements
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
for j in range(12):
    X1 = np.array(X)
    X2 = []
    for i in range(len(X1)):
        X2.append(X1[i].transpose())

    X2_1 = []
    for i in range(len(X2)):
        X2_1.append(X2[i][j])
        
    X_train, X_test, y_train, y_test = train_test_split(X2_1, Y, test_size=0.2, random_state=42)
    
    model = create_model()
    
    # Reshape the data to match the input shape expected by the model
    X_train = np.array(X_train).reshape(-1, 1000, 1)
    X_test = np.array(X_test).reshape(-1, 1000, 1)

    # Train the model
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'\nTest loss: {loss}\nTest accuracy: {accuracy}')

    model.save(f'trial_{j+1}_model.h5')

    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tfmodel = converter.convert()
    open(f"ftlite_model/tflite_lead_{j+1}.tflite","wb").write(tfmodel)

    print(f"model {j+1} saved")
    
print("Models created sucessfully")

Epoch 1/20
546/546 ━━━━━━━━━━━━━━━━━━━━ 253s 455ms/step - accuracy: 0.3400 - loss: 0.5830 - val_accuracy: 0.4139 - val_loss: 0.5263
Epoch 2/20
546/546 ━━━━━━━━━━━━━━━━━━━━ 261s 453ms/step - accuracy: 0.4324 - loss: 0.5276 - val_accuracy: 0.4629 - val_loss: 0.4707
Epoch 3/20
546/546 ━━━━━━━━━━━━━━━━━━━━ 259s 446ms/step - accuracy: 0.4702 - loss: 0.4828 - val_accuracy: 0.5311 - val_loss: 0.4400
Epoch 4/20
546/546 ━━━━━━━━━━━━━━━━━━━━ 259s 440ms/step - accuracy: 0.5337 - loss: 0.4473 - val_accuracy: 0.5311 - val_loss: 0.4249
Epoch 5/20
546/546 ━━━━━━━━━━━━━━━━━━━━ 259s 435ms/step - accuracy: 0.5393 - loss: 0.4368 - val_accuracy: 0.5449 - val_loss: 0.4215
Epoch 6/20
546/546 ━━━━━━━━━━━━━━━━━━━━ 262s 434ms/step - accuracy: 0.5445 - loss: 0.4299 - val_accuracy: 0.5321 - val_loss: 0.4406
Epoch 7/20
546/546 ━━━━━━━━━━━━━━━━━━━━ 267s 442ms/step - accuracy: 0.5449 - loss: 0.4257 - val_accuracy: 0.5579 - val_loss: 0.4101
Epoch 8/20
546/546 ━━━━━━━━━━━━━━━━━━━━ 243s 444ms/step - accuracy: 0.5486 -


Test loss: 0.4154638350009918
Test accuracy: 0.5627289414405823
INFO:tensorflow:Assets written to: C:\Users\John\AppData\Local\Temp\tmphhy6urjn\assets


INFO:tensorflow:Assets written to: C:\Users\John\AppData\Local\Temp\tmphhy6urjn\assets


Saved artifact at 'C:\Users\John\AppData\Local\Temp\tmphhy6urjn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1000, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2617515112432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461525488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461536576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461532528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461537280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461534992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461537456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461537808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461537632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461984944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461538


Test loss: 0.3841567933559418
Test accuracy: 0.6165292859077454
INFO:tensorflow:Assets written to: C:\Users\John\AppData\Local\Temp\tmp8stsd0dw\assets


...

INFO:tensorflow:Assets written to: C:\Users\John\AppData\Local\Temp\tmpkyg4e7om\assets


Saved artifact at 'C:\Users\John\AppData\Local\Temp\tmpkyg4e7om'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1000, 1), dtype=tf.float32, name='keras_tensor_76')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2617894091440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617894092496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461413088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461408512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461414848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461412208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461416608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461413616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461416080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461416960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617461


Test loss: 0.4290865957736969
Test accuracy: 0.5812729001045227
INFO:tensorflow:Assets written to: C:\Users\John\AppData\Local\Temp\tmphbk76f1p\assets


INFO:tensorflow:Assets written to: C:\Users\John\AppData\Local\Temp\tmphbk76f1p\assets


Saved artifact at 'C:\Users\John\AppData\Local\Temp\tmphbk76f1p'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1000, 1), dtype=tf.float32, name='keras_tensor_95')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  2617515718112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2617515719168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2618352816144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2618352814032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2618352817904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2618352815264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2618352819664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2618352816672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2618352819136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2618352820016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2618352